## Imports


In [32]:
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.tools import DuckDuckGoSearchRun

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain_openai import OpenAIEmbeddings

from langchain.tools.retriever import create_retriever_tool

from langchain.utils.math import cosine_similarity
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from openai import OpenAI
from dotenv import load_dotenv


## Pegando .env api

In [33]:
load_dotenv() 
client = OpenAI()

## loadando modelo

In [34]:

model = ChatOpenAI(
    model='gpt-3.5-turbo-1106',
    temperature=0.7
)

## loaders e splitting

In [35]:

loader_web = WebBaseLoader("https://www.metro.sp.gov.br/")
docs = loader_web.load()
    
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

## definindo db

In [36]:
embedding = OpenAIEmbeddings()
vectorStore = FAISS.from_documents(splitter.split_documents(docs), embedding=OpenAIEmbeddings())
retriever = vectorStore.as_retriever(search_kwargs={"k": 3})

## Templates

In [37]:
system_rotas_template = """
        Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
        Você é um especialista no sistema de Metrô de São Paulo em rotas. \
        Você tem amplo conhecimento sobre as linhas do metrô, suas interligações e horários de pico, fornecendo alternativas rápidas e eficientes para quem utiliza o metrô. \
        Responda em ate 15 palavras, com informações claras e precisas.

    """

system_horarios_template = """
        Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
        Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo. \
        Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô. \
        Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço. \
        Responda em ate 15 palavras, com informações claras e precisas.
 

"""

system_politicas_template = """
        Você só resposnde perguntas relacionadas ao metrô de São Paulo. \
        Você é um especialista em políticas públicas relacionadas ao sistema de Metrô de São Paulo. \
        Você tem conhecimento detalhado sobre as leis, regulamentos e iniciativas que afetam o metrô, e oferece respostas claras \
        e baseadas em fatos sobre como essas políticas impactam os passageiros. \
        Responda em ate 15 palavras, com informações claras e precisas.

        """



## Exemplos de perguntas pra semantic routing

In [38]:
rotas_perguntas = [
    "Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?",
    "Como posso evitar as estações mais lotadas durante o horário de pico?",
    "Quais são as melhores conexões entre as linhas azul e verde do metrô?",
]

horarios_perguntas = [
    "Há alguma greve no Metrô de São Paulo programada para esta semana?",
    "O metrô está funcionando normalmente durante o feriado?",
    "Quais são os horários de operação do metrô aos domingos?",
]

politicas_perguntas = [
    "Quais são as políticas de acessibilidade nas estações do Metrô de São Paulo?",
    "Como as tarifas do metrô são decididas e ajustadas?",
    "Quais são as políticas de sustentabilidade adotadas pelo Metrô de São Paulo?",
]


## Fazendo embeddings com os exemplos pra calcular qual o cosseno e poder classificar a dúvida

In [39]:
embeddings = OpenAIEmbeddings()

rotas_perguntas_embeddings = embeddings.embed_documents(rotas_perguntas)
horarios_perguntas_embeddings = embeddings.embed_documents(horarios_perguntas)
politicas_perguntas_embeddings = embeddings.embed_documents(politicas_perguntas)

## LLM routing com busca semântica

In [40]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])

    rotas_similarity = cosine_similarity([query_embedding], rotas_perguntas_embeddings)[0]

    horarios_similarity = cosine_similarity([query_embedding], horarios_perguntas_embeddings)[0]

    politicas_similarity = cosine_similarity([query_embedding], politicas_perguntas_embeddings)[0]

    max_similarity = max(
        max(rotas_similarity), max(horarios_similarity), max(politicas_similarity)
    )

    if max_similarity == max(rotas_similarity):
        template = system_rotas_template,
    elif max_similarity == max(horarios_similarity):
        template=system_horarios_template
    else:
        template=system_politicas_template
    return template

## gerando prmpt a depender do output do routing

In [41]:
#"Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?"
pergunta = "QUal a rota do metro consolacao até o aeroporto de guarulhos"

template = str(prompt_router({"query": pergunta}))
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder("chat_history"),
        ("user", "{input}"),
    ]
)
chain = prompt | model


## instancia RedisChatMessageHistory 

In [42]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_redis import RedisChatMessageHistory

import os
REDIS_URL ="redis://localhost:6379"
history = RedisChatMessageHistory(session_id="123213", redis_url=REDIS_URL)
def get_redis_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, redis_url=REDIS_URL)



## runnable

In [43]:
chain_with_history = RunnableWithMessageHistory(
    chain, get_redis_history, input_messages_key="input", history_messages_key="chat_history"
)



In [53]:
pergunta = str(input("pergunta: "))
while True:
    
    
    response1 = chain_with_history.invoke(
        
        {"input": pergunta},
        config={"configurable": {"session_id": "1121"}},
    )

    print(response1.content)
    pergunta = str(input("pergunta: "))


É verdade, meu chapa, tem que fazer baldeação na estação Sé para seguir até a Consolação. Mas é de boa, rapidinho você faz essa troca e segue o rolê!


KeyboardInterrupt: Interrupted by user